In [1]:
df = spark.read.\
option("head", None).\
option("inferSchema", True).\
csv("../Documents/src/flask-genesis/genesis/data/krk/krk.data",)

In [2]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: integer (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: integer (nullable = true)
 |-- _c6: string (nullable = true)



In [50]:
[c[0] for c in df[df.columns].dtypes if c[1] == 'int']

['_c1', '_c3', '_c5']

In [4]:
def get_indexer_input(data):
    str_cols_value = {}
    for c, t in data[data.columns].dtypes:
        if t == 'string':
            str_cols_value[c] = StringIndexer(inputCol=c, outputCol='indexed_' + c).fit(data)
    return str_cols_value

In [5]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

indexer_input = get_indexer_input(df)
data_test, data_train = df.randomSplit(weights=[0.3, 0.7], seed=10)

In [6]:
indexer_input.keys()

dict_keys(['_c0', '_c2', '_c4', '_c6'])

In [ ]:
x_cols = list(set(data_train.columns) - set(indexer_input.keys())) + str_ind_cols - ['indexed__c6']

In [ ]:
str_ind_cols = ['indexed_' + column for column in indexer_input.keys() if column != '_c6']
x_cols = list(set(data_train.columns) - set(indexer_input.keys())) + str_ind_cols
print (x_cols)

In [7]:
from pyspark.ml import Pipeline
from pyspark.mllib.tree import RandomForest
from pyspark.ml.regression import GBTRegressor

str_ind_cols = ['indexed_' + column for column in indexer_input.keys() if column != '_c6']
x_cols = list(set(data_train.columns) - set(indexer_input.keys())) + str_ind_cols
print (x_cols)
# x_cols.append('_c6')
indexers = indexer_input.values()
pipeline_tr = Pipeline(stages=indexers)
data_tr = pipeline_tr.fit(data_train).transform(data_train)
data_tr = data_tr.drop('_c0', '_c2', '_c4', '_c6')
data_tr.show()
assembler = VectorAssembler(inputCols=x_cols, outputCol="features")
# rfc = RandomForest.trainClassifier(trainingData, numClasses=18, categoricalFeaturesInfo={},
#                                      numTrees=3, featureSubsetStrategy="auto",
#                                      impurity='gini', maxDepth=4, maxBins=32)
# gbt = GBTRegressor(featuresCol="features", labelCol="indexed__c6", stepSize=0.008, maxDepth=5, subsamplingRate=0.75,
#                        seed=10, maxIter=500, minInstancesPerNode=5, checkpointInterval=100, maxBins=64)
# pipeline_training = Pipeline(stages=[assembler, gbt])
# model = pipeline_training.fit(data_tr)


['_c5', '_c3', '_c1', 'indexed__c0', 'indexed__c2', 'indexed__c4']
+---+---+---+-----------+-----------+-----------+-----------+
|_c1|_c3|_c5|indexed__c0|indexed__c2|indexed__c4|indexed__c6|
+---+---+---+-----------+-----------+-----------+-----------+
|  1|  2|  1|        3.0|        4.0|        7.0|        9.0|
|  1|  2|  2|        3.0|        4.0|        7.0|        5.0|
|  1|  2|  1|        3.0|        4.0|        5.0|        8.0|
|  1|  2|  3|        3.0|        4.0|        5.0|        5.0|
|  1|  2|  1|        3.0|        4.0|        3.0|        9.0|
|  1|  2|  2|        3.0|        4.0|        3.0|        0.0|
|  1|  2|  4|        3.0|        4.0|        3.0|        5.0|
|  1|  2|  1|        3.0|        4.0|        2.0|        8.0|
|  1|  2|  2|        3.0|        4.0|        2.0|        1.0|
|  1|  2|  3|        3.0|        4.0|        2.0|        5.0|
|  1|  2|  4|        3.0|        4.0|        2.0|        5.0|
|  1|  2|  5|        3.0|        4.0|        2.0|        5.0|
|  

In [51]:
data_tr[data_tr.columns].dtypes

[('_c1', 'int'),
 ('_c3', 'int'),
 ('_c5', 'int'),
 ('indexed__c0', 'double'),
 ('indexed__c2', 'double'),
 ('indexed__c4', 'double'),
 ('indexed__c6', 'double')]

In [58]:
temp_data_tr = data_tr.select(*(col(c).cast("float").alias(c) if c in [d[0] for d in data_tr[data_tr.columns].dtypes if d[1] == 'int'] else col(c) for c in data_tr.columns))

In [59]:
temp_data_tr.show()

+---+---+---+-----------+-----------+-----------+-----------+
|_c1|_c3|_c5|indexed__c0|indexed__c2|indexed__c4|indexed__c6|
+---+---+---+-----------+-----------+-----------+-----------+
|1.0|2.0|1.0|        3.0|        4.0|        7.0|        9.0|
|1.0|2.0|2.0|        3.0|        4.0|        7.0|        5.0|
|1.0|2.0|1.0|        3.0|        4.0|        5.0|        8.0|
|1.0|2.0|3.0|        3.0|        4.0|        5.0|        5.0|
|1.0|2.0|1.0|        3.0|        4.0|        3.0|        9.0|
|1.0|2.0|2.0|        3.0|        4.0|        3.0|        0.0|
|1.0|2.0|4.0|        3.0|        4.0|        3.0|        5.0|
|1.0|2.0|1.0|        3.0|        4.0|        2.0|        8.0|
|1.0|2.0|2.0|        3.0|        4.0|        2.0|        1.0|
|1.0|2.0|3.0|        3.0|        4.0|        2.0|        5.0|
|1.0|2.0|4.0|        3.0|        4.0|        2.0|        5.0|
|1.0|2.0|5.0|        3.0|        4.0|        2.0|        5.0|
|1.0|2.0|1.0|        3.0|        4.0|        1.0|        9.0|
|1.0|2.0

In [70]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors


transformed = assembler.transform(temp_data_tr)
lp_transformed = transformed.select(col("indexed__c6").alias("label"), col("features"))\
.rdd\
.map(lambda x: LabeledPoint(x.label, as_mllib(x.features)))

In [71]:
lp_transformed.take(2)

[LabeledPoint(9.0, [1.0,2.0,1.0,3.0,4.0,7.0]),
 LabeledPoint(5.0, [2.0,2.0,1.0,3.0,4.0,7.0])]

In [69]:
from pyspark.ml import linalg as ml_linalg

def as_mllib(v):
    if isinstance(v, ml_linalg.SparseVector):
        return MLLibVectors.sparse(v.size, v.indices, v.values)
    elif isinstance(v, ml_linalg.DenseVector):
        return MLLibVectors.dense(v.toArray())
    else:
        raise TypeError("Unsupported type: {0}".format(type(v)))